<a href="https://colab.research.google.com/github/KunalKatariya/Lyrics-Generator/blob/master/Word_Level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q keras

In [0]:
!pip install -q numpy

In [0]:
!pip install -q pandas

In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [5]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam, RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import os
import sys
import io
import re
import random

Using TensorFlow backend.


In [6]:
!wget https://raw.githubusercontent.com/ivan-liljeqvist/ailyrics/master/corpus.txt -P "/content/drive/My Drive/app"

--2019-01-04 07:03:09--  https://raw.githubusercontent.com/ivan-liljeqvist/ailyrics/master/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7709143 (7.4M) [text/plain]
Saving to: ‘/content/drive/My Drive/app/corpus.txt.2’

corpus.txt.2        100%[===================>]   7.35M  --.-KB/s    in 0.08s   

2019-01-04 07:03:09 (92.2 MB/s) - ‘/content/drive/My Drive/app/corpus.txt.2’ saved [7709143/7709143]



In [0]:
df = pd.read_csv("/content/drive/My Drive/app/corpus.txt.1")

In [8]:
with io.open("/content/drive/My Drive/app/corpus.txt.1", encoding = 'utf-8') as f:
  text = f.read().lower().replace('\n', ' \n ')
print("Corpus Length in characters: ", len(text))

text_in_words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print('Corpus length in words:', len(text_in_words))

Corpus Length in characters:  7719341
Corpus length in words: 1608123


In [9]:
#Calculating frequency of words
MIN_WORD_FREQUENCY = 10
word_freq = {}
for word in text_in_words:
  word_freq[word] = word_freq.get(word, 0) + 1
  
ignored_word = set()
for k, v in word_freq.items():
  if word_freq[k] < MIN_WORD_FREQUENCY:
    ignored_word.add(k)

words = set(text_in_words)
print("Unique words before ignoring:", len(words))
print("Ignoring words with frequency <", MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_word)
print("Unique words after ignoring:", len(words))

Unique words before ignoring: 42129
Ignoring words with frequency < 10
Unique words after ignoring: 5536


In [0]:
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

In [0]:
step = 1
sequence_len = 10
sentences = []
next_words = []
ignored = 0

In [12]:
for i in range(0, len(text_in_words) - sequence_len, step):
  # Only add the sequences where no word is in ignored_words
  if len(set(text_in_words[i: i+sequence_len+1]).intersection(ignored_word)) == 0:
    sentences.append(text_in_words[i: i + sequence_len])
    next_words.append(text_in_words[i + sequence_len])
  else:
    ignored = ignored + 1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

Ignored sequences: 586279
Remaining sequences: 1021834


In [14]:
print('Shuffling sentences')

tmp_sentences = []
tmp_next_word = []
for i in np.random.permutation(len(sentences)):
  tmp_sentences.append(sentences[i])
  tmp_next_word.append(next_words[i])

cut_index = int(len(sentences) * (1.-(2/100.)))
x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

print("Size of training set = %d" % len(x_train))
print("Size of test set = %d" % len(y_test))

Shuffling sentences
Size of training set = 1001397
Size of test set = 20437
